In [1]:
from bids.layout import BIDSLayout
import pandas as pd
from nipype.interfaces.base import Bunch

/home/data/nbc/data-analysis/env/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data_dir = '/home/data/nbc/external-datasets/ds001491/'
layout = BIDSLayout(data_dir)
sub = sorted(layout.get_subjects())[0]
#for sub in layout.get_subjects():
evs = layout.get(task='images', suffix='events', subject=sub)[0]
df = pd.read_csv(evs.path, sep='\t')
im_file = layout.get(subject=sub, task='images', extensions='nii.gz')[0].path
preproc_file = ('/home/data/nbc/external-datasets/ds001491/derivatives/afni-step1/'
                'sub-01/func/sub-01_task-images_echo-1_desc-realign_bold.nii.gz')
t_r = layout.get_metadata(im_file)['RepetitionTime']

In [3]:
onsets, durations, amplitudes = [], [], []
condnames = ['Image', 'Rating']

for tt in condnames:
    tt_df = df.loc[df['trial_action'] == tt]
    onsets.append(tt_df['onset'].tolist())
    durations.append(tt_df['duration'].tolist())
    amplitudes.append(tt_df['intensity_response'].tolist())

subject_info = Bunch(conditions=condnames, onsets=onsets, durations=durations)

In [4]:
from nipype.algorithms import modelgen
from nipype.interfaces.nipy import model
s = modelgen.SpecifyModel()
s.inputs.input_units = 'secs'
s.inputs.functional_runs = [preproc_file]
s.inputs.time_repetition = t_r
s.inputs.subject_info = [subject_info]
s.inputs.high_pass_filter_cutoff = 128.
out = s.run()

In [5]:
fit = model.FitGLM()
fit.inputs.TR = t_r
fit.inputs.session_info = out.outputs.session_info
fit.inputs.plot_design_matrix = True
fit.inputs.save_residuals = True

In [6]:
out2 = fit.run()

TypeError: __init__() takes exactly 2 arguments (1 given)